In [984]:
import pandas as pd
import numpy as np
import datetime
import math
from copy import deepcopy

Path = "G:/.shortcut-targets-by-id/1sZS2kL5L66960IYJ4xfNEV9CuKrs_7fE/1조/RAW DATA"

In [985]:
# df = pd.read_csv(Path+"/월별기상관측(2000.01-2021.12).csv",encoding="cp949")
# df.insert(1,'행정구역',object)

# df.to_csv(Path+"/수정_월별기상관측.csv")

In [986]:
# df = pd.read_csv(Path+"/수정_월별기상관측.csv")
# # df.head()

# # 기상 측정소 위치 라벨링
# df.loc[df['지점명']=='서울', '행정구역'] = '서울특별시'
# df.loc[df['지점명']=='무안', '행정구역'] = '전라남도'
# df.loc[df['지점명']=='진도(첨찰산)', '행정구역'] = '전라남도'
# df.loc[df['지점명']=='대구(기)', '행정구역'] = '대구광역시'
# df.loc[df['지점명']=='주암', '행정구역'] = '전라남도'
# df.loc[df['지점명']=='성산포', '행정구역'] = '제주도'

In [987]:
# df_new[df_new["행정구역"]=="<class 'object'>"]["지점명"].unique()
# df.loc[df['지점명']=='주암']
# df.to_csv(Path+"/최종_월별기상관측.csv",encoding="cp949")

In [988]:
#1. 최대일사량 결측치 처리: 동일년도+동일월+동일 행정구역 데이터의 중앙값으로 대체 ex)강원도의 철원지역 결측치 2021년 1월 강원도의 최대일사량
#    (1) 세종시 : 기간 2000년 01월~2019년 04월 충청북도 최대일사량 중앙값으로 대체
 

In [989]:
# df = pd.read_csv(Path+"/최종_월별기상관측.csv",encoding="cp949")

In [990]:
# df.drop(df.columns[:2],axis=1,inplace=True)   # 0,1번째 컬럼 drop
# # df.head()

In [991]:
# 필요 컬럼만 추출('행정구역', "지점명", '일시', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)', '월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)')
# df_new=df[['행정구역', "지점명", '일시', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)', '월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']]

In [992]:
# df_ulsan = df_new[df_new["행정구역"]=="울산광역시"][["행정구역", "합계 일사량(MJ/m2)"]]

# print(len(df_ulsan))
# print(df_ulsan["합계 일사량(MJ/m2)"].isnull().sum())

# 울산광역시의 일사량은 전부 NaN 값

In [993]:
# df_new["합계 일사량(MJ/m2)"].replace(0.0, np.nan, inplace=True)   # 일사량 0.0을 전부 NaN으로 변경

In [994]:
# df_new[df_new["합계 일사량(MJ/m2)"]==0.0]["합계 일사량(MJ/m2)"]

In [995]:
# 일시와 행정구역에 따른 합계 일사량 중앙값
# sunshine = pd.pivot_table(df_new, index="일시", columns="행정구역", values="합계 일사량(MJ/m2)",aggfunc="median")

In [996]:
# sunshine.interpolate(inplace=True)  # 부산광역시 18년도 8월 결측치를 앞뒤 값으로 보간

In [997]:
# df2 = df_new

In [998]:
# df["합계 일사량(MJ/m2)"].fillna()
# for i in range(len(df2)) : 
#     dt = df2.loc[i,"일시"]
#     dstr = df2.loc[i,"행정구역"]
# #     print(dt)
# #     print(dstr)
#     if np.isnan(df2.loc[i,"합계 일사량(MJ/m2)"]) :  # 일사량이 NaN일 때
#         if df2.loc[i,"행정구역"] == "세종특별자치시" :       # 행정구역이 세종시라면
#             df2.loc[i,"합계 일사량(MJ/m2)"] = sunshine["충청북도"][dt]     # 동일 일시의 충청북도 중앙값을 채움
#         elif df2.loc[i,"행정구역"] == "울산광역시" :        # 행정구역이 울산광역시라면
#             df2.loc[i,"합계 일사량(MJ/m2)"] = sunshine["경상남도"][dt]     # 동일 일시의 경상남도 중앙값을 채움
#         else : 
#             df2.loc[i,"합계 일사량(MJ/m2)"] = sunshine[dstr][dt]   # 동일 일시, 동일 행정 구역의 중앙값을 채움

# # pd.pivot_table(df2, index="일시", columns="행정구역", values="합계 일사량(MJ/m2)",aggfunc="median")

In [999]:
# df2[df2["행정구역"]=="세종특별자치시"][["일시","합계 일사량(MJ/m2)"]]

In [1000]:
# df2[df2["행정구역"]=="충청북도"][["일시","합계 일사량(MJ/m2)"]]

In [1001]:
# df2[df2["합계 일사량(MJ/m2)"].isnull()]
# ["울산광역시", "세종특별자치시"]

In [1002]:
# avr_tem = pd.pivot_table(df_new, index="일시", columns="행정구역", values="평균기온(°C)",aggfunc="median")

In [1003]:
# high_tem = pd.pivot_table(df_new, index="일시", columns="행정구역", values="최고기온(°C)",aggfunc="median")

In [1004]:
# low_tem = pd.pivot_table(df_new, index="일시", columns="행정구역", values="최저기온(°C)",aggfunc="median")

In [1005]:
# df2.info()

In [1006]:
# def fill_tem(temperature, dataframe, pivot) : 
#     for i in range(len(dataframe)) : 
#         dt = dataframe.loc[i,"일시"]
#         dstr = dataframe.loc[i,"행정구역"]

#         if np.isnan(dataframe.loc[i,temperature]) :  # 기온 관련 컬럼값이 NaN일 때 
#             dataframe.loc[i,temperature] = pivot[dstr][dt]   # 동일 일시, 동일 행정 구역의 중앙값을 채움
    
#     return dataframe

In [1007]:
# df2 = fill_tem("최저기온(°C)", df2, low_tem)
# df2 = fill_tem("최고기온(°C)", df2, high_tem)
# df2 = fill_tem("평균기온(°C)", df2, avr_tem)

In [1008]:
# df2_tst[df2_tst["평균기온(°C)"].isnull()]

In [1009]:
# df2['평균기온(°C)'] = df2['평균기온(°C)'].fillna(value=avr_tem["충청북도"]["2019-05"])

In [1010]:
# df2.info()

In [1011]:
# df2[df2["행정구역"]=="세종특별자치시"][["일시","합계 일사량(MJ/m2)"]]

In [1012]:
# df2[df2["행정구역"]=="충청북도"][["일시","합계 일사량(MJ/m2)"]]

In [1013]:
# df2_test=pd.pivot_table(df2, index="일시", columns="행정구역", values="합계 일사량(MJ/m2)",aggfunc="median")

In [1014]:
# df2_test

In [1015]:
# chch_copy = df2[df2['행정구역']=='충청북도']
# chch_copy

In [1016]:
# chch_median = chch_copy.groupby("일시").median()

In [1017]:
# chch_median.insert(0,column="행정구역", value="세종특별자치시")
# chch_median.insert(1,column="지점명", value="세종")
# # chch_median

In [1018]:
# chch_median.reset_index(inplace=True)
# chch_median

In [1019]:
# chch_median.insert(3,column="일시",value=chch_median["일시"])
# chch_median
# chch_median.columns

In [1020]:
# chch_median = chch_median[['행정구역', '지점명', '일시', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)','월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']]
# chch_median

In [1021]:
# idx_chch = chch_median[chch_median["일시"]=="2019-05"].index[0]
# idx_chch

In [1022]:
# chch_bef_201905 = chch_median.iloc[:idx_chch,:]
# # chch_bef_201905

In [1023]:
# df2[df2["행정구역"]=="세종특별자치시"]

In [1024]:
# new_df = pd.concat([df2.iloc[:14984],chch_bef_201905,df2.iloc[14984:]])
# # new_df

In [1025]:
# new_df[new_df["행정구역"]=="세종특별자치시"]

In [1026]:
# new_df.info()

In [1027]:
# new_df[new_df['행정구역']=='세종특별자치시'].isnull().sum()

In [1028]:
# new_df.fillna(0.0,inplace=True)
# new_df.info()

In [1029]:
# new_df.to_csv("data/최종_최종__월별기상관측.csv",encoding="cp949")

In [1030]:
# new_df.groupby(["행정구역", "일시"]).median()

In [1031]:
new_df = pd.read_csv("./data/최종_최종__월별기상관측.csv", encoding="cp949")
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23078 entries, 0 to 23077
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          23078 non-null  int64  
 1   행정구역                23078 non-null  object 
 2   지점명                 23078 non-null  object 
 3   일시                  23078 non-null  object 
 4   평균기온(°C)            23078 non-null  float64
 5   최고기온(°C)            23078 non-null  float64
 6   최저기온(°C)            23078 non-null  float64
 7   월합강수량(00~24h만)(mm)  23078 non-null  float64
 8   합계 일사량(MJ/m2)       23078 non-null  float64
dtypes: float64(5), int64(1), object(3)
memory usage: 1.6+ MB


In [1032]:
new_df.drop("Unnamed: 0", axis=1, inplace=True)  # 0번째 컬럼 drop
# new_df.head(3)

In [1033]:
new_df["일시"] = pd.to_datetime(new_df["일시"])  # 일시를 datetime으로 변경
# new_df.head()

In [1034]:
new_df["연도"] =  new_df["일시"].dt.year   # 연도만 추출
new_df["월"] =  new_df["일시"].dt.month    # 월만 추출
# new_df.head(10)

In [1035]:
# new_df.columns

In [1036]:
# 컬럼 순서 변경
new_df = new_df[['행정구역', '지점명', '연도', '월', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)','월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']]
# new_df.head(3)


In [1037]:
# test = new_df.groupby(["연도","행정구역","월"]).median()[:24]

In [1038]:
# test

In [1039]:
# test2 = test.iloc[[0, 12]]
# for i in range(1, int(len(test)/2)) : 
#     a = test.iloc[[i,i+12]]
#     # print(a)
#     test2 = pd.merge(left = test2, right = a, left_index = True, right_index = True, how = "outer")
# test2

In [1040]:
# test2 = test.iloc[0]
# for i in range(1, len(test)) : 
#     a = test.iloc[i]
#     test2 = pd.merge(left = test2, right = a, left_index = True, right_index = True, how = "outer") 
# test2

In [1041]:
# i = 0, j = 0, k = 0, new_col = 0 empty_frame.iloc[0,0] = test.iloc[0,0]
#               k = 1, new_col = 1 empty_frame.iloc[0,1] = test.iloc[0,1]
#               k = 2, new_col = 2 empty_frame.iloc[0,2] = test.iloc[0,2]
#               k = 3, new_col = 3 empty_frame.iloc[0,3] = test.iloc[0,3]
#               k = 4, new_col = 4 empty_frame.iloc[0,4] = test.iloc[0,4]
# i = 1, j = 0, k = 0, new_col = 5 empty_frame.iloc[0,5] = test.iloc[1,0]
#               k = 1, new_col = 6 empty_frame.iloc[0,6] = test.iloc[1,1]
#               k = 2, new_col = 7 empty_frame.iloc[0,7] = test.iloc[1,2]
#               k = 3, new_col = 8 empty_frame.iloc[0,8] = test.iloc[1,3]
#               k = 4, new_col = 9 empty_frame.iloc[0,9] = test.iloc[1,4]
# i = 12, j = 1, k = 0, new_col = 60 empty_frame.iloc[0,5] = test.iloc[1,0]
#               k = 1, new_col = 6 empty_frame.iloc[0,6] = test.iloc[1,1]
#               k = 2, new_col = 7 empty_frame.iloc[0,7] = test.iloc[1,2]
#               k = 3, new_col = 8 empty_frame.iloc[0,8] = test.iloc[1,3]
#               k = 4, new_col = 9 empty_frame.iloc[0,9] = test.iloc[1,4]

In [1042]:
real_test = new_df.groupby(["연도","행정구역","월"]).median() # 연도, 행정구역, 월별로 group화

In [1043]:
empty_array = np.empty((int(len(real_test)/12),60))   # (4488/12, 60)의 구조를 가지는 빈 행렬을 생성
empty_array[:] = np.nan
empty_frame = pd.DataFrame(empty_array)

cname = []
for i in range(1,13) : 
    L = ['평균기온(°C)', '최고기온(°C)', '최저기온(°C)','월합강수량(00~24h만)(mm)', '합계 일사량(MJ/m2)']
    for elem in L : 
        word = elem + "_{}월".format(i)  # 각 컬럼명에 1월부터 12월까지 붙여줌
        cname.append(word)

empty_frame.columns = cname   # 변수에 월수가 붙은 리스트를 컬럼명으로 대체

In [1044]:
# cname

In [1045]:
for i in range(len(real_test)) : 
    j = math.floor(i/12)  # 각 행 인덱스를 12로 나눈 후 내림  >>>  0부터 11까지는 0으로, 12부터 23까지는 1로.....
    remain = i%12  # 각 행 인덱스를 12로 나눈 나머지  >>> 0과 12, 24는 0으로, 1과 13 25는 1로......
    for k in range(len(real_test.iloc[i])) : 
        new_col = (remain*5)+k      # remain이 0이면 new_col은 0부터 4까지, remain이 1이면 new_col은 5부터 9까지, remain이 11이면 new_col은 55부터 59까지, 
        empty_frame.iloc[j,new_col] = real_test.iloc[i,k]  # real_test의 데이터를 empty_frame에 삽입
wanna_frame = empty_frame

In [1046]:
b = real_test.index   # groupby한 데이터 프레임의 인덱스를 저장
a = [(b[i][0],b[i][1]) for i in range(len(b)) if i%12==0]   # 인덱스를 12로 나눈 후 나머지가 없는 행만 추출(연도는 같고 월만 다르기 때문에 그 중 처음만 가져온다)
                                                            # 그 후 연도와 행정구역만 튜플로 저장

In [1047]:
# len(a)

In [1048]:
new_index = pd.MultiIndex.from_tuples(a)  # 연도와 행정구역을 추출한 튜플 리스트를 멀티인덱스로 변환

In [1049]:
wanna_frame.index = new_index
# wanna_frame.head()

In [1050]:
# city 배열 생성
city = ["서울특별시", "부산광역시", "대구광역시", "인천광역시", "광주광역시", "대전광역시", "울산광역시", "세종특별자치시", "경기도", "강원도", "충청북도", "충청남도", "전라북도", "전라남도", "경상북도", "경상남도", "제주도"]
city_list= ["seoul", "busan", "daegu", "incheon", "gwangju", "daejeon", "ulsan", "sejong", "gyeonggi", "gangwon", "chungbuk", "chungnam", "jeonbuk", "jeonnam", "gyeongbuk", "gyeongnam", "jeju"]

# season 배열 생성
season = ["spring", "summer", "fall", "winter"]

# .query()를 사용하여 계절별로 데이터 프레임 생성 

# 지역별, 계절별 배추 생산량 csv 불러오기
for i in range(len(city)):
    for j in range(len(season)):
        globals()[city_list[i] + "_" + season[j]] = pd.read_csv("./data/"+city_list[i]+"_"+season[j]+".csv")
        # globals()[city_list[i] + "_" + season[j]] = new_df[(new_df["행정구역"]==city[i]) & (new_df["일시"].isin(season_list[j]))]
        # globals()[city_list[i] + "_" + season[j]].reset_index(drop=True, inplace=True)
        # globals()[city_list[i] + "_" + season[j]] = globals()[city_list[i] + "_" + season[j]].drop(["행정구역"], axis=1)
        # globals()[city_list[i] + "_" + season[j]] = globals()[city_list[i] + "_" + season[j]].rename(columns={"지점명": "지역명"})
        # # 지역명 컬럼 삭제
        # globals()[city_list[i] + "_" + season[j]] = globals()[city_list[i] + "_" + season[j]].drop(["지역명"], axis=1)
        # # 일시 데이터를 년도로 변경
        # globals()[city_list[i] + "_" + season[j]]["일시"] = globals()[city_list[i] + "_" + season[j]]["일시"].str[:4]
        # # csv 파일로 저장
        # globals()[city_list[i] + "_" + season[j]].to_csv("data/weather/" + city_list[i] + "_" + season[j] + ".csv", encoding="utf-8", index=False)

In [1051]:
# seoul_fall_select

In [1052]:
for i in range(len(city)):
    for j in range(len(season)):
        k_list = []
        for k in range(len(globals()[city_list[i] + "_" + season[j]].iloc[0])) : 
            if k % 3 == 2 or k % 3 == 0 : 
                k_list.append(k)
        
        globals()[city_list[i] + "_" + season[j]+ "_select"] = globals()[city_list[i] + "_" + season[j]].iloc[:,k_list]

In [1053]:
year_list_spring = []
for i in range(2000,2022) : 
    year_list_spring.append((i,"일반봄배추:면적 (ha)"))
    for k in range(i,i+1) : 
        year_list_spring.append((k,"생산량 (톤)"))

In [1054]:
year_list_summer = []
for i in range(2000,2022) : 
    year_list_summer.append((i,"고랭지배추:면적 (ha)"))
    for k in range(i,i+1) : 
        year_list_summer.append((k,"생산량 (톤)"))

In [1055]:
year_list_fall = []
for i in range(2000,2022) : 
    year_list_fall.append((i,"노지가을배추:면적 (ha)"))
    for k in range(i,i+1) : 
        year_list_fall.append((k,"생산량 (톤)"))
# year_list

In [1056]:
year_list_winter = []
for i in range(2014,2022) : 
    year_list_winter.append((i,"노지겨울배추:면적 (ha)"))
    for k in range(i,i+1) : 
        year_list_winter.append((k,"생산량 (톤)"))

In [1057]:
# year_list = [(i,i) for i in range(2000,2022)]
# year_list

In [1058]:
year_list_spring = pd.MultiIndex.from_tuples(year_list_spring)
year_list_summer = pd.MultiIndex.from_tuples(year_list_summer)
year_list_fall = pd.MultiIndex.from_tuples(year_list_fall)
year_list_winter = pd.MultiIndex.from_tuples(year_list_winter)
# year_list

In [1059]:
# for i in range(len(city)):
#     for j in range(len(season)):
#         print(len(globals()[city_list[i] + "_" + season[j]+ "_select"].iloc[0]))

In [1060]:
for i in range(len(city)):
    for j in range(len(season)):
        # if city[i] == "세종특별자치시"
        globals()[city_list[i] + "_" + season[j]+ "_select"] = globals()[city_list[i] + "_" + season[j]+ "_select"].transpose()
        globals()[city_list[i] + "_" + season[j]+ "_select"].index = globals()["year_list_"+season[j]]
        # globals()[city_list[i] + "_" + season[j]+ "_select"] = globals()[city_list[i] + "_" + season[j]+ "_select"].reindex(globals()["year_list_"+season[j]])
        # 겨울은 16개다


In [1061]:
# jeonnam_winter_select.drop([0],axis=1).unstack().droplevel(0, axis=1)

In [1062]:
for i in range(len(city)):
    for j in range(len(season)):
        globals()[city_list[i] + "_" + season[j]+ "_select"] = globals()[city_list[i] + "_" + season[j]+ "_select"].drop([0],axis=1).unstack().droplevel(0, axis=1)

In [1063]:
for i in range(len(season)) : 
    wan_idx = wanna_frame.index  
    globals()["idx_"+season[i]] = [(wan_idx[j][0],season[i],wan_idx[j][1]) for j in range(len(wan_idx))]
    globals()["idx_"+season[i]] = pd.MultiIndex.from_tuples(globals()["idx_"+season[i]])

In [1064]:
for i in range(len(season)) : 
    globals()["wanna_frame_"+season[i]] = deepcopy(wanna_frame)
    globals()["wanna_frame_"+season[i]].index = globals()["idx_"+season[i]]

In [1065]:
for i in range(len(season)) : 
    globals()["wanna_frame_"+season[i]] = globals()["wanna_frame_"+season[i]].swaplevel(0).sort_index(level=0)

In [1066]:
# # wanna_frame = 
# wanna_frame = pd.concat([wanna_frame_spring, wanna_frame_summer, wanna_frame_fall, wanna_frame_winter], axis=0)

In [1067]:
# wanna_frame_winter

In [1068]:
# gangwon_spring_select

In [1069]:
new_city = dict(zip(city,city_list))
new_city = dict(sorted(new_city.items()))
# new_city

In [1070]:
cities_spring = []
cities_summer = []
cities_fall = []
cities_winter = []

for k,v in new_city.items():
    for j in range(len(season)):
        globals()["cities_"+season[j]].append(new_city[k]+"_"+season[j]+"_select")

In [1071]:
# cities_spring

In [1072]:
concat_spring = pd.concat([gangwon_spring_select,
gyeonggi_spring_select,
gyeongnam_spring_select,
gyeongbuk_spring_select,
 gwangju_spring_select,
 daegu_spring_select,
 daejeon_spring_select,
 busan_spring_select,
 seoul_spring_select,
 sejong_spring_select,
 ulsan_spring_select,
 incheon_spring_select,
 jeonnam_spring_select,
 jeonbuk_spring_select,
 jeju_spring_select,
 chungnam_spring_select,
 chungbuk_spring_select])

In [1073]:
concat_summer = pd.concat([gangwon_summer_select,
gyeonggi_summer_select,
gyeongnam_summer_select,
gyeongbuk_summer_select,
 gwangju_summer_select,
 daegu_summer_select,
 daejeon_summer_select,
 busan_summer_select,
 seoul_summer_select,
 sejong_summer_select,
 ulsan_summer_select,
 incheon_summer_select,
 jeonnam_summer_select,
 jeonbuk_summer_select,
 jeju_summer_select,
 chungnam_summer_select,
 chungbuk_summer_select])

In [1074]:
concat_fall = pd.concat([gangwon_fall_select,
gyeonggi_fall_select,
gyeongnam_fall_select,
gyeongbuk_fall_select,
gwangju_fall_select,
daegu_fall_select,
daejeon_fall_select,
busan_fall_select,
seoul_fall_select,
sejong_fall_select,
ulsan_fall_select,
incheon_fall_select,
jeonnam_fall_select,
jeonbuk_fall_select,
jeju_fall_select,
chungnam_fall_select,
chungbuk_fall_select])


In [1075]:
concat_winter = pd.concat([gangwon_winter_select,
gyeonggi_winter_select,
gyeongnam_winter_select,
gyeongbuk_winter_select,
gwangju_winter_select,
daegu_winter_select,
daejeon_winter_select,
busan_winter_select,
seoul_winter_select,
sejong_winter_select,
ulsan_winter_select,
incheon_winter_select,
jeonnam_winter_select,
jeonbuk_winter_select,
jeju_winter_select,
chungnam_winter_select,
chungbuk_winter_select])


In [1076]:
idx_winter_from2014 = idx_winter[238:]
idx_winter_from2014

MultiIndex([(2014, 'winter',     '강원도'),
            (2014, 'winter',     '경기도'),
            (2014, 'winter',    '경상남도'),
            (2014, 'winter',    '경상북도'),
            (2014, 'winter',   '광주광역시'),
            (2014, 'winter',   '대구광역시'),
            (2014, 'winter',   '대전광역시'),
            (2014, 'winter',   '부산광역시'),
            (2014, 'winter',   '서울특별시'),
            (2014, 'winter', '세종특별자치시'),
            ...
            (2021, 'winter',   '부산광역시'),
            (2021, 'winter',   '서울특별시'),
            (2021, 'winter', '세종특별자치시'),
            (2021, 'winter',   '울산광역시'),
            (2021, 'winter',   '인천광역시'),
            (2021, 'winter',    '전라남도'),
            (2021, 'winter',    '전라북도'),
            (2021, 'winter',     '제주도'),
            (2021, 'winter',    '충청남도'),
            (2021, 'winter',    '충청북도')],
           length=136)

In [1077]:
for i in range(len(season)-1) : 
    globals()["concat_"+season[i]].index = globals()["idx_"+season[i]]

In [1078]:
concat_winter.index = idx_winter_from2014
concat_winter

노지겨울배추:면적 (ha) 생산량 (톤)
2014 winter 강원도                0       0
            경기도                0       0
            경상남도               0       0
            경상북도               0       0
            광주광역시              0       0
...                          ...     ...
2021 winter 전라남도               0       0
            전라북도               0       0
            제주도                0       0
            충청남도               0       0
            충청북도               0       0

[136 rows x 2 columns]

In [1079]:
# for i in range(len(idx_winter)) : 
#     if idx_winter[i][0] == 2014 : 
#         print(i)
#         break

In [1080]:
for i in range(len(season)) : 
    globals()["concat_"+season[i]] = globals()["concat_"+season[i]].replace('-',np.nan)
    globals()["concat_"+season[i]] = globals()["concat_"+season[i]].astype("float")


In [1081]:
for i in range(len(season)) : 
    globals()["concat_"+season[i]] = globals()["concat_"+season[i]].swaplevel(0).sort_index(level=0)

col_list_spring = concat_spring.columns[::-1].to_list()
concat_spring = concat_spring[col_list_spring]

for i in range(len(season)) : 
    globals()["concat_"+season[i]].rename(columns={globals()["concat_"+season[i]].columns[0]:"면적 (ha)"},inplace=True)

In [1082]:
# print(concat_spring.columns)
# print(concat_summer.columns)
# print(concat_fall.columns)
# print(concat_winter.columns)

In [1083]:
for i in range(len(season)) : 
    globals()["wanna_frame_"+season[i]] = pd.concat([globals()["wanna_frame_"+season[i]], globals()["concat_"+season[i]]],axis=1)

In [1084]:
df_jinhyeok = pd.concat([wanna_frame_spring,wanna_frame_summer,wanna_frame_fall,wanna_frame_winter])

In [1087]:
df_jinhyeok.tail()

평균기온(°C)_1월  최고기온(°C)_1월  최저기온(°C)_1월  \
충청북도 winter 2017         -2.5         12.8        -16.0   
            2018         -4.3          8.8        -19.2   
            2019         -2.2          8.6        -13.1   
            2020          1.0         12.3         -9.7   
            2021         -3.1         13.3        -19.6   

                  월합강수량(00~24h만)(mm)_1월  합계 일사량(MJ/m2)_1월  평균기온(°C)_2월  \
충청북도 winter 2017                   16.4           276.060         -0.2   
            2018                   17.6           281.440         -2.4   
            2019                    1.0           301.040          0.5   
            2020                   68.1           239.400          1.8   
            2021                   18.2           285.645          1.8   

                  최고기온(°C)_2월  최저기온(°C)_2월  월합강수량(00~24h만)(mm)_2월  \
충청북도 winter 2017         13.8        -11.9                   38.7   
            2018         13.0        -17.4                   30.6   
            2019         13.2        -11.6                   29.0   
            2020         15.6        -12.7                   62.7   
            2021         22.1        -11.9                   12.3   

                  합계 일사량(MJ/m2)_2월  ...  최저기온(°C)_11월  월합강수량(00~24h만)(mm)_11월  \
충청북도 winter 2017           361.545  ...          -7.7                    17.2   
            2018           342.500  ...          -5.6                    50.0   
            2019           340.155  ...          -5.9                    84.6   
            2020           338.925  ...          -5.5                    37.0   
            2021           359.560  ...          -5.2                    44.9   

                  합계 일사량(MJ/m2)_11월  평균기온(°C)_12월  최고기온(°C)_12월  최저기온(°C)_12월  \
충청북도 winter 2017            290.585          -2.3          10.3         -14.5   
            2018            281.060          -1.5          14.6         -14.8   
            2019            275.295           0.3          13.7         -11.0   
            2020            290.085          -2.0          11.6         -17.0   
            2021            280.140          -0.1          12.6         -14.7   

                  월합강수량(00~24h만)(mm)_12월  합계 일사량(MJ/m2)_12월  면적 (ha)   생산량 (톤)  
충청북도 winter 2017                    25.5            248.975      0.0       0.0  
            2018                    30.0            261.265      0.0       0.0  
            2019                    27.2            242.825   3307.0  223797.0  
            2020                     5.6            267.930     50.0    3310.0  
            2021                     5.6            249.960      0.0       0.0  

[5 rows x 62 columns]

In [1088]:
# df_jinhyeok.to_csv("./data/진혁_데이터프레임.csv")

In [1089]:
# df = pd.read_csv("./data/진혁_데이터프레임.csv")
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1496 entries, 0 to 1495
Data columns (total 65 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1496 non-null   object 
 1   Unnamed: 1              1496 non-null   object 
 2   Unnamed: 2              1496 non-null   int64  
 3   평균기온(°C)_1월             1496 non-null   float64
 4   최고기온(°C)_1월             1496 non-null   float64
 5   최저기온(°C)_1월             1496 non-null   float64
 6   월합강수량(00~24h만)(mm)_1월   1496 non-null   float64
 7   합계 일사량(MJ/m2)_1월        1496 non-null   float64
 8   평균기온(°C)_2월             1496 non-null   float64
 9   최고기온(°C)_2월             1496 non-null   float64
 10  최저기온(°C)_2월             1496 non-null   float64
 11  월합강수량(00~24h만)(mm)_2월   1496 non-null   float64
 12  합계 일사량(MJ/m2)_2월        1496 non-null   float64
 13  평균기온(°C)_3월             1496 non-null   float64
 14  최고기온(°C)_3월             1496 non-null   

In [1090]:
# df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,평균기온(°C)_1월,최고기온(°C)_1월,최저기온(°C)_1월,월합강수량(00~24h만)(mm)_1월,합계 일사량(MJ/m2)_1월,평균기온(°C)_2월,최고기온(°C)_2월,...,최저기온(°C)_11월,월합강수량(00~24h만)(mm)_11월,합계 일사량(MJ/m2)_11월,평균기온(°C)_12월,최고기온(°C)_12월,최저기온(°C)_12월,월합강수량(00~24h만)(mm)_12월,합계 일사량(MJ/m2)_12월,면적 (ha),생산량 (톤)
0,강원도,spring,2000,-2.9,8.5,-16.0,48.1,152.990,-3.3,7.7,...,-8.4,30.2,231.670,-1.4,12.5,-14.9,16.9,234.160,2147.0,87619.0
1,강원도,spring,2001,-6.1,7.3,-23.5,37.2,257.045,-2.2,13.5,...,-7.4,15.5,269.430,-3.0,10.6,-13.7,18.5,256.155,842.0,47271.0
2,강원도,spring,2002,-2.8,10.6,-19.2,60.8,251.510,0.3,14.2,...,-9.8,7.5,281.405,-1.3,16.0,-14.5,49.9,220.680,196.0,9230.0
3,강원도,spring,2003,-5.4,6.8,-20.0,23.7,260.055,-0.4,10.8,...,-7.6,71.7,210.580,-1.5,12.5,-14.1,8.5,260.870,520.0,24752.0
4,강원도,spring,2004,-4.4,8.5,-18.3,11.1,294.320,0.1,18.7,...,-4.9,39.5,267.470,-0.3,15.1,-13.0,20.5,254.615,2491.0,109380.0
